# XGBoost ile Boosting

İkinci proje olan regresyon projesinde kullandığınız veri kümesini tekrar kullanarak XGBoost algoritmasını kullanarak regresyon ve sınıflandırma modelleri oluşturun ve bu model için en uygun parametre değerlerini belirleyin. Projelerinizdeki modellerle karşılaştırın.

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
import scipy.stats as stats
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error,explained_variance_score
from sklearn.metrics import mean_squared_error as mse
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import log_loss
from sklearn.utils import resample
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb

In [33]:
life_df=pd.read_csv("Life Expectancy Data.csv")
life_df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [35]:
life_df2=life_df.copy()

In [36]:
life_df2.columns.values

array(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure',
       'Hepatitis B', 'Measles ', ' BMI ', 'under-five deaths ', 'Polio',
       'Total expenditure', 'Diphtheria ', ' HIV/AIDS', 'GDP',
       'Population', ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'], dtype=object)

Words have space

In [37]:
life_df2.columns = life_df2.columns.str.strip()

In [38]:
life_df2.columns.values

array(['Country', 'Year', 'Status', 'Life expectancy', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure',
       'Hepatitis B', 'Measles', 'BMI', 'under-five deaths', 'Polio',
       'Total expenditure', 'Diphtheria', 'HIV/AIDS', 'GDP', 'Population',
       'thinness  1-19 years', 'thinness 5-9 years',
       'Income composition of resources', 'Schooling'], dtype=object)

Removed the space

In [39]:
pd.get_dummies(life_df2["Status"])

,Developed,Developing
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
2933,0,1
2934,0,1
2935,0,1
2936,0,1


In [40]:
life_df2= pd.concat([life_df2, pd.get_dummies(life_df["Status"], drop_first=True)], axis=1)
life_df2.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Developing
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,1
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,1
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,1
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,1


## Missing Value

In [41]:
life_df2.isnull().sum()

Country                              0
Year                                 0
Status                               0
Life expectancy                     10
Adult Mortality                     10
infant deaths                        0
Alcohol                            194
percentage expenditure               0
Hepatitis B                        553
Measles                              0
BMI                                 34
under-five deaths                    0
Polio                               19
Total expenditure                  226
Diphtheria                          19
HIV/AIDS                             0
GDP                                448
Population                         652
thinness  1-19 years                34
thinness 5-9 years                  34
Income composition of resources    167
Schooling                          163
Developing                           0
dtype: int64

In [42]:
life_df3=life_df2.copy()

In [43]:
life_df3["Life expectancy"]=life_df3["Life expectancy"].fillna(life_df3.groupby("Country")["Life expectancy"].transform("mean"))
life_df3["Life expectancy"]=life_df3["Life expectancy"].fillna(life_df3["Life expectancy"].mean())

life_df3["Adult Mortality"]=life_df3["Adult Mortality"].fillna(life_df3.groupby("Country")["Adult Mortality"].transform("mean"))
life_df3["Adult Mortality"]=life_df3["Adult Mortality"].fillna(life_df3["Adult Mortality"].mean())

life_df3["Alcohol"]=life_df3["Alcohol"].fillna(life_df3.groupby("Country")["Alcohol"].transform("mean"))
life_df3["Alcohol"]=life_df3["Alcohol"].fillna(life_df3["Alcohol"].mean())

life_df3["Hepatitis B"]=life_df3["Hepatitis B"].fillna(life_df3.groupby("Country")["Hepatitis B"].transform("mean"))
life_df3["Hepatitis B"]=life_df3["Hepatitis B"].fillna(life_df3["Hepatitis B"].mean())

life_df3["BMI"]=life_df3["BMI"].fillna(life_df3.groupby("Country")["BMI"].transform("mean"))
life_df3["BMI"]=life_df3["BMI"].fillna(life_df3["BMI"].mean())

life_df3["Polio"]=life_df3["Polio"].fillna(life_df3.groupby("Country")["Polio"].transform("mean"))
life_df3["Polio"]=life_df3["Polio"].fillna(life_df3["Polio"].mean())

life_df3["Total expenditure"]=life_df3["Total expenditure"].fillna(life_df3.groupby("Country")["Total expenditure"].transform("mean"))
life_df3["Total expenditure"]=life_df3["Total expenditure"].fillna(life_df3["Total expenditure"].mean())

life_df3["Diphtheria"]=life_df3["Diphtheria"].fillna(life_df3.groupby("Country")["Diphtheria"].transform("mean"))
life_df3["Diphtheria"]=life_df3["Diphtheria"].fillna(life_df3["Diphtheria"].mean())

life_df3["GDP"]=life_df3["GDP"].fillna(life_df3.groupby("Country")["GDP"].transform("mean"))
life_df3["GDP"]=life_df3["GDP"].fillna(life_df3["GDP"].mean())

life_df3["Population"]=life_df3["Population"].fillna(life_df3.groupby("Country")["Population"].transform("mean"))
life_df3["Population"]=life_df3["Population"].fillna(life_df3["Population"].mean())

life_df3["thinness  1-19 years"]=life_df3["thinness  1-19 years"].fillna(life_df3.groupby("Country")["thinness  1-19 years"].transform("mean"))
life_df3["thinness  1-19 years"]=life_df3["thinness  1-19 years"].fillna(life_df3["thinness  1-19 years"].mean())

life_df3["thinness 5-9 years"]=life_df3["thinness 5-9 years"].fillna(life_df3.groupby("Country")["thinness 5-9 years"].transform("mean"))
life_df3["thinness 5-9 years"]=life_df3["thinness 5-9 years"].fillna(life_df3["thinness 5-9 years"].mean())

life_df3["Income composition of resources"]=life_df3["Income composition of resources"].fillna(life_df3.groupby("Country")["Income composition of resources"].transform("mean"))
life_df3["Income composition of resources"]=life_df3["Income composition of resources"].fillna(life_df3["Income composition of resources"].mean())

life_df3["Schooling"]=life_df3["Schooling"].fillna(life_df3.groupby("Country")["Schooling"].transform("mean"))
life_df3["Schooling"]=life_df3["Schooling"].fillna(life_df3["Schooling"].mean())

In [44]:
life_df3.isnull().sum()

Country                            0
Year                               0
Status                             0
Life expectancy                    0
Adult Mortality                    0
infant deaths                      0
Alcohol                            0
percentage expenditure             0
Hepatitis B                        0
Measles                            0
BMI                                0
under-five deaths                  0
Polio                              0
Total expenditure                  0
Diphtheria                         0
HIV/AIDS                           0
GDP                                0
Population                         0
thinness  1-19 years               0
thinness 5-9 years                 0
Income composition of resources    0
Schooling                          0
Developing                         0
dtype: int64

## 1. Classification

**Train and Test Data**

In [45]:
y_xgb = life_df3['Developing']
X_xgb = life_df3.drop(['Developing', 'Year', 'Status', 'Country'], axis=1)

In [46]:
X_xgb_train, X_xgb_test, y_xgb_train, y_xgb_test =  train_test_split(X_xgb, y_xgb, test_size=0.20, random_state=111)

**Dmatrix**

In [47]:
dtrain_xgb = xgb.DMatrix(X_xgb_train, label=y_xgb_train)
dtest_xgb = xgb.DMatrix(X_xgb_test, label=y_xgb_test)

**Parametres**

In [48]:
params = {"objective":"reg:logistic", "max_depth":3}

**Train Model**

In [49]:
bst = xgb.XGBClassifier(**params)
bst.fit(X_xgb_train, y_xgb_train)

C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='reg:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

**Prediction**

In [50]:
y_pred_xgb_train = bst.predict(X_xgb_train)
y_pred_xgb_test = bst.predict(X_xgb_test)

In [51]:
print("R-squared of the model in training set is: {}".format(bst.score(X_xgb_train, y_xgb_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(bst.score(X_xgb_test, y_xgb_test)))


R-squared of the model in training set is: 1.0
-----Test set statistics-----
R-squared of the model in test set is: 0.9812925170068028


**Performance Measuring**

In [52]:
y_predict_xgb = bst.predict(X_xgb_test)
print("XGB Accuracy        : {:.6f}".format(accuracy_score(y_xgb_test, y_pred_xgb_test)))

XGB Accuracy        : 0.981293


In [53]:
print(classification_report(y_xgb_test,  y_pred_xgb_test))

print("f1_score() value        : {:.2f}".format(f1_score(y_xgb_test, y_pred_xgb_test)))
print("recall_score() value    : {:.2f}".format(recall_score(y_xgb_test, y_pred_xgb_test)))
print("precision_score() value : {:.2f}".format(precision_score(y_xgb_test, y_pred_xgb_test)))
print('\n')

metrics = precision_recall_fscore_support(y_xgb_test, y_pred_xgb_test)
print("Precision  :" , metrics[0]) 
print("Recall     :" , metrics[1]) 
print("F1 Score   :" , metrics[2]) 

              precision    recall  f1-score   support

           0       0.95      0.94      0.94        96
           1       0.99      0.99      0.99       492

    accuracy                           0.98       588
   macro avg       0.97      0.96      0.97       588
weighted avg       0.98      0.98      0.98       588

f1_score() value        : 0.99
recall_score() value    : 0.99
precision_score() value : 0.99


Precision  : [0.94736842 0.98782961]
Recall     : [0.9375    0.9898374]
F1 Score   : [0.94240838 0.98883249]


**KFold - Cross Validation**

**Train**

In [54]:
cvscores_2 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv =  2 ---> Average cross validation score :  0.9825531914893617


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv =  3 ---> Average cross validation score :  0.9851071015890391


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

...

C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv =  6 ---> Average cross validation score :  0.9863847712998243


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv = 7 ---> Average cross validation score  :  0.9876585551744788


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [55]:
cvscores_2 = cross_val_score(bst, X_xgb_test, y_xgb_test, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(bst, X_xgb_test, y_xgb_test, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(bst, X_xgb_test, y_xgb_test, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(bst, X_xgb_test, y_xgb_test, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(bst, X_xgb_test, y_xgb_test, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(bst, X_xgb_test, y_xgb_test, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv =  2 ---> Average cross validation score :  0.9608843537414966
cv =  3 ---> Average cross validation score :  0.947278911564626


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv =  4 ---> Average cross validation score :  0.9523809523809523


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

...

C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv =  6 ---> Average cross validation score :  0.9523809523809524


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv = 7 ---> Average cross validation score  :  0.9422222222222223


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## 2. Regression

**Train and Test Data**

In [56]:
y_xgb2 = life_df3['Life expectancy']
X_xgb2 = life_df3.drop(['Life expectancy', 'Status', 'Country'], axis=1)

In [57]:
X_xgb2_train, X_xgb2_test, y_xgb2_train, y_xgb2_test =  train_test_split(X_xgb2, y_xgb2, test_size=0.20, random_state=111)

**Dmatrix**

In [58]:
dtrain_xgb2 = xgb.DMatrix(X_xgb2_train, label=y_xgb2_train)
dtest_xgb2 = xgb.DMatrix(X_xgb2_test, label=y_xgb2_test)

**Train Model**

In [59]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_xgb2_train, y_xgb2_train)

[23:10:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=8, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=10, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

**Prediction**

In [60]:
y_pred_xgb2_train = xg_reg.predict(X_xgb2_train)
y_pred_xgb2_test = xg_reg.predict(X_xgb2_test)

In [61]:
print("R-squared of the model in training set is: {}".format(xg_reg.score(X_xgb2_train, y_xgb2_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(xg_reg.score(X_xgb2_test, y_xgb2_test)))
print("Mean absolute error (MAE) of the prediction is: {}".format(mean_absolute_error(y_xgb2_test, y_pred_xgb2_test)))
print("Mean squared error (MSE) of the prediction is: {}".format(mse(y_xgb2_test, y_pred_xgb2_test)))
print("Root mean squared error (RMSE) of the prediction is: {}".format(rmse(y_xgb2_test, y_pred_xgb2_test)))
print("Mean absolute percentage error (MAPE) of the prediction is: {}".format(np.mean(np.abs((y_xgb2_test - y_pred_xgb2_test) / y_xgb2_test)) * 100))

R-squared of the model in training set is: -5.654887960869511
-----Test set statistics-----
R-squared of the model in test set is: -5.90836477522416
Mean absolute error (MAE) of the prediction is: 23.991956821026303
Mean squared error (MSE) of the prediction is: 601.4090298510251
Root mean squared error (RMSE) of the prediction is: 24.52364226315139
Mean absolute percentage error (MAPE) of the prediction is: 34.357096204507


**KFold - Cross Validation**

**Train**

In [62]:
cvscores_2 = cross_val_score(xg_reg, X_xgb2_train, y_xgb2_train, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(bst, X_xgb_train, y_xgb_train, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

[23:10:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:10:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
cv =  2 ---> Average cross validation score :  -5.718668456167544


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv =  3 ---> Average cross validation score :  0.9851071015890391


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv =  4 ---> Average cross validation score :  0.9868103118589855


...

cv =  5 ---> Average cross validation score :  0.9851063829787232


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv =  6 ---> Average cross validation score :  0.9863847712998243


C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use 

cv = 7 ---> Average cross validation score  :  0.9876585551744788


**Train**

In [63]:
cvscores_2 = cross_val_score(xg_reg, X_xgb2_test, y_xgb2_test, cv=2)
print('cv =  2 ---> Average cross validation score : ', np.mean(cvscores_2))

cvscores_3 = cross_val_score(xg_reg, X_xgb2_test, y_xgb2_test, cv=3)
print('cv =  3 ---> Average cross validation score : ', np.mean(cvscores_3))

cvscores_4 = cross_val_score(xg_reg, X_xgb2_test, y_xgb2_test, cv=4)
print('cv =  4 ---> Average cross validation score : ', np.mean(cvscores_4))

cvscores_5 = cross_val_score(xg_reg, X_xgb2_test, y_xgb2_test, cv=5)
print('cv =  5 ---> Average cross validation score : ', np.mean(cvscores_5))

cvscores_6 = cross_val_score(xg_reg, X_xgb2_test, y_xgb2_test, cv=6)
print('cv =  6 ---> Average cross validation score : ', np.mean(cvscores_6))

cvscores_7 = cross_val_score(xg_reg, X_xgb2_test, y_xgb2_test, cv=15)
print('cv = 7 ---> Average cross validation score  : ', np.mean(cvscores_7))

[23:11:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:11:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
cv =  2 ---> Average cross validation score :  -6.233247827968731
[23:11:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:11:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[23:11:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
cv =  3 ---> Average cross validation score :  -6